**Notas generales.**

Modelos:

Tanto gpt-oss como qwen hacen uso de la estrategia de generación en formato harmony. Es decir, primero tienen una sección interna de razonamiento (un tanto esquizofrénico) PREVIO a la respuesta global. Entre más ambiguo sea el prompt más larga es esta sección, por eso prompts como `prompt4` tienen tan buenos resultados. El modelo es capaz de inferir que se le está pasando un ejemplo y logra seguir el formato de buena manera.


`max_new_tokens` Rates:
1. gpt-oss: ~2000 ya empieza a dar resultados buenos, pero no siempre genera la última premisa completa.
2. qwen: ~8000 ya genera la respuesta final

Al final en ambos casos se deja este parámetro en 8000 para estandarizar.
 
Obs: 8000 todavía le genera problemas a gpt. Las últimas premisas tiende a darlas incompletas. Para la siguiente ronda de experimentos usaremos 10k para la traducción.

In [1]:
import torch, re
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(dev)
print(torch.cuda.get_device_name())

path = r'/home/flopezp/Kurosagol/FOLIO/FOLIO/folio_test.jsonl'
dataset = pd.read_json(path, lines = True)
dataset.head()

cuda
NVIDIA RTX 5000 Ada Generation


,story_id,premises,premises-FOL,conclusion,conclusion-FOL,label,example_id
0,324,No professional soccer players play profession...,∀x ((Professional(x) ∧ SoccerPlayer(x)) → ¬Pla...,Stephen Curry is an athlete.,Athlete(stephenCurry),Uncertain,828
1,324,No professional soccer players play profession...,∀x ((Professional(x) ∧ SoccerPlayer(x)) → ¬Pla...,Stephen Curry is an athlete and a professional...,Athlete(stephenCurry) ∧ Professional(stephenCu...,False,829
2,324,No professional soccer players play profession...,∀x ((Professional(x) ∧ SoccerPlayer(x)) → ¬Pla...,"If Stephen Curry is a professional centerback,...",(Professional(stephenCurry) ∧ CenterBack(steph...,True,830
3,357,If people join student government in high scho...,"∀x (Join(x, studentGovernment) → ManyOpportuni...",Tyler joined the student government in high sc...,"JoinIn(tyler, studentGovernment, highSchool)",Uncertain,947
4,357,If people join student government in high scho...,"∀x (Join(x, studentGovernment) → ManyOpportuni...",If Tyler either both stays alone in high schoo...,"¬(StayAloneIn(tyler, highSchool) ⊕ (Well-adjus...",True,948


In [8]:
path = r'/home/flopezp/Kurosagol/FOLIO/FOLIO/folio_train.jsonl'
dataset2 = pd.read_json(path, lines = True)
print(dataset2['premises-FOL'][69])
print('----')
print(dataset2['conclusion-FOL'][69])
print(dataset2['conclusion-FOL'][68])
print(dataset2['conclusion-FOL'][67])
print('----')
print(dataset2['label'][69])
print(dataset2['label'][68])
print(dataset2['label'][67])

print("==============")
print(dataset2['premises-FOL'][113])
print('----')
print(dataset2['conclusion-FOL'][113])
#print(dataset2['conclusion-FOL'][68])
#print(dataset2['conclusion-FOL'][67])
print('----')
print(dataset2['label'][113])
#print(dataset2['label'][68])
#print(dataset2['label'][67])

∀x (WorkAt(x, meta) → HighIncome(x))
∀x (HighIncome(x) → ¬MeansToDestination(x, bus))
∀x (MeansToDestination(x, bus) ⊕ MeansToDestination(x, drive))
∀x (HaveCar(x) → MeansToDestination(x, drive))
∀x (Student(x) → ¬ MeansToDestination(x, drive))
HaveCar(james) ∨ WorkAt(james, meta)
----
MeansToDestination(x, drive) ∨ Student(james)
Student(james)
¬HighIncome(james)
----
True
False
Uncertain
English(thomasBarber) ∧ ProfessionalFootballer(thomasBarber)
PlayedFor(thomasBarber, astonVilla) ∧ PlayedIn(astonVilla,theFootballLeague)
PlayedAs(thomasBarber, halfBack) ∧ PlayedAs(thomasBarber, insideLeft)
ScoredTheWinningGoalIn(thomasBarber, facupfinal1913)
----
∃x (English(x) ∧ ProfessionalFootballer(x) ∧ ScoredTheWinningGoalIn(x, facupfinal1913))
----
True


In [ ]:
# HuggingFace download
#folio_nice = load_dataset('fol-autoformalization/FOLIO')

In [3]:
# Models

gpt_tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")
gpt_model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b", device_map = 'cuda')

# -------------------------------
qwen_quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype= torch.bfloat16)
qwen_tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-8B')
qwen_model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen3-8B', quantization_config = qwen_quant_config, device_map = 'cuda')

Fetching 41 files:   0%|          | 0/41 [00:00<?, ?it/s]

_masked_compaction.py:   0%|          | 0.00/814 [00:00<?, ?B/s]

matmul_ogs.py: 0.00B [00:00, ?B/s]

__init__.cpython-312.pyc:   0%|          | 0.00/220 [00:00<?, ?B/s]

_finalize_matmul.py: 0.00B [00:00, ?B/s]

...

_topk_forward.py: 0.00B [00:00, ?B/s]

_topk_backward.py: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Fetching 41 files:   0%|          | 0/41 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

### Logic Prompts (Igual para todos los checkpoints)

#### Translation

In [4]:
prompt = """
    Translate the following premises from natural language to first order logic. 
    PREMISES:
    All eels are fish.
    No fish are plants.
    Everything displayed in the collection is either a plant or an animal.
    All multicellular animals are not bacteria.
    All animals displayed in the collection are multicellular.
    A sea eel is displayed in the collection.
    The sea eel is an eel or an animal or not a plant.
"""

prompt2 = """
    Given a set of premises description and a question, the task is to parse the problem into first-order logic formulars.
    --------------
    Premises:
    All people who regularly drink coffee are dependent on caffeine. 
    People either regularly drink coffee or joke about being addicted to caffeine. 
    No one who jokes about being addicted to caffeine is unaware that caffeine is a drug. 
    Rina is either a student and unaware that caffeine is a drug, or neither a student nor unaware that caffeine is a drug. 
    If Rina is not a person dependent on caffeine and a student, then Rina is either a person dependent on caffeine and a student, or neither a person dependent on caffeine nor a student.
    Predicates:
    Dependent(x) 
    Drinks(x) 
    Jokes(x) 
    Unaware(x)
    Student(x)
    Premises:
    ∀x (Drinks(x) → Dependent(x)) 
    ∀x (Drinks(x) ⊕ Jokes(x))
    ∀x (Jokes(x) → ¬Unaware(x)) 
    (Student(rina) ∧ Unaware(rina)) ⊕ ¬(Student(rina) ∨ Unaware(rina)) 
    ¬(Dependent(rina) ∧ Student(rina)) → (Dependent(rina) ∧ Student(rina)) ⊕ ¬(Dependent(rina) ∨ Student(rina)) 
    --------------
    
    Premises:
    All eels are fish.
    No fish are plants.
    Everything displayed in the collection is either a plant or an animal.
    All multicellular animals are not bacteria.
    All animals displayed in the collection are multicellular.
    A sea eel is displayed in the collection.
    The sea eel is an eel or an animal or not a plant.
    Predicates:
"""


prompt3 = """
    Given a set of premises, the task is to parse the problem and the question into first-order logic formulars.
    The grammar of the first-order logic formular is defined as follows:
    1) logical conjunction of expr1 and expr2: expr1 ∧ expr2
    2) logical disjunction of expr1 and expr2: expr1 ∨ expr2
    3) logical exclusive disjunction of expr1 and expr2: expr1 ⊕ expr2
    4) logical negation of expr1: ¬expr1
    5) expr1 implies expr2: expr1 → expr2
    6) expr1 if and only if expr2: expr1 ↔ expr2
    7) logical universal quantification: ∀x
    8) logical existential quantification: ∃x
    --------------
    Premises:
    All people who regularly drink coffee are dependent on caffeine. People either regularly drink coffee or joke about being addicted to caffeine. No one who jokes about being addicted to caffeine is unaware that caffeine is a drug. Rina is either a student and unaware that caffeine is a drug, or neither a student nor unaware that caffeine is a drug. If Rina is not a person dependent on caffeine and a student, then Rina is either a person dependent on caffeine and a student, or neither a person dependent on caffeine nor a student.
    Predicates:
    Dependent(x) ::: x is a person dependent on caffeine.
    Drinks(x) ::: x regularly drinks coffee.
    Jokes(x) ::: x jokes about being addicted to caffeine.
    Unaware(x) ::: x is unaware that caffeine is a drug.
    Student(x) ::: x is a student.
    Premises:
    ∀x (Drinks(x) → Dependent(x)) ::: All people who regularly drink coffee are dependent on caffeine.
    ∀x (Drinks(x) ⊕ Jokes(x)) ::: People either regularly drink coffee or joke about being addicted to caffeine.
    ∀x (Jokes(x) → ¬Unaware(x)) ::: No one who jokes about being addicted to caffeine is unaware that caffeine is a drug. 
    (Student(rina) ∧ Unaware(rina)) ⊕ ¬(Student(rina) ∨ Unaware(rina)) ::: Rina is either a student and unaware that caffeine is a drug, or neither a student nor unaware that caffeine is a drug. 
    ¬(Dependent(rina) ∧ Student(rina)) → (Dependent(rina) ∧ Student(rina)) ⊕ ¬(Dependent(rina) ∨ Student(rina)) ::: If Rina is not a person dependent on caffeine and a student, then Rina is either a person dependent on caffeine and a student, or neither a person dependent on caffeine nor a student.
    --------------
    
    Premises:
    All eels are fish.
    No fish are plants.
    Everything displayed in the collection is either a plant or an animal.
    All multicellular animals are not bacteria.
    All animals displayed in the collection are multicellular.
    A sea eel is displayed in the collection.
    The sea eel is an eel or an animal or not a plant.
    Predicates:
"""

final_prompt = """
    Given a set of premises, the task is to parse the problem and the question into first-order logic formulars. Answer only with the translated premises.
    The grammar of the first-order logic formular is defined as follows:
    1) logical conjunction of expr1 and expr2: expr1 ∧ expr2
    2) logical disjunction of expr1 and expr2: expr1 ∨ expr2
    3) logical exclusive disjunction of expr1 and expr2: expr1 ⊕ expr2
    4) logical negation of expr1: ¬expr1
    5) expr1 implies expr2: expr1 → expr2
    6) expr1 if and only if expr2: expr1 ↔ expr2
    7) logical universal quantification: ∀x
    8) logical existential quantification: ∃x
    --------------
    Natural Language Premises:
    All people who regularly drink coffee are dependent on caffeine. People either regularly drink coffee or joke about being addicted to caffeine. No one who jokes about being addicted to caffeine is unaware that caffeine is a drug. Rina is either a student and unaware that caffeine is a drug, or neither a student nor unaware that caffeine is a drug. If Rina is not a person dependent on caffeine and a student, then Rina is either a person dependent on caffeine and a student, or neither a person dependent on caffeine nor a student.
    Predicates:
    Dependent(x) ::: x is a person dependent on caffeine.
    Drinks(x) ::: x regularly drinks coffee.
    Jokes(x) ::: x jokes about being addicted to caffeine.
    Unaware(x) ::: x is unaware that caffeine is a drug.
    Student(x) ::: x is a student.
    FOL Premises:
    ∀x (Drinks(x) → Dependent(x)) ::: All people who regularly drink coffee are dependent on caffeine.
    ∀x (Drinks(x) ⊕ Jokes(x)) ::: People either regularly drink coffee or joke about being addicted to caffeine.
    ∀x (Jokes(x) → ¬Unaware(x)) ::: No one who jokes about being addicted to caffeine is unaware that caffeine is a drug. 
    (Student(rina) ∧ Unaware(rina)) ⊕ ¬(Student(rina) ∨ Unaware(rina)) ::: Rina is either a student and unaware that caffeine is a drug, or neither a student nor unaware that caffeine is a drug. 
    ¬(Dependent(rina) ∧ Student(rina)) → (Dependent(rina) ∧ Student(rina)) ⊕ ¬(Dependent(rina) ∨ Student(rina)) ::: If Rina is not a person dependent on caffeine and a student, then Rina is either a person dependent on caffeine and a student, or neither a person dependent on caffeine nor a student.
    --------------
    
    Natural Language Premises:
    {}
    FOL Premises:
"""

#### Inference

In [ ]:
prompt = """
    Given the following premises in first order logic, determine wether the conclusion is True, False, or Uncertain. 
    PREMISES:
    ∀x (WorkAt(x, meta) → HighIncome(x))
    ∀x (HighIncome(x) → ¬MeansToDestination(x, bus))
    ∀x (MeansToDestination(x, bus) ⊕ MeansToDestination(x, drive))
    ∀x (HaveCar(x) → MeansToDestination(x, drive))
    ∀x (Student(x) → ¬ MeansToDestination(x, drive))
    HaveCar(james) ∨ WorkAt(james, meta)
    CONCLUSION:
    MeansToDestination(x, drive) ∨ Student(james)
"""

prompt2 = """
    Given a set of premises and a conclusion in first order logic, determine wether the conclusion is True, False, or Uncertain.
    A True conclusion is one that can be inferred using only the premises.
    A False conclusion is one that contradicts the premises at some point.
    An Uncertain conclusion is one that can't be inferred but doesn't contradict the premises.
    --------------
    PREMISES:
    ∀x (WorkAt(x, meta) → HighIncome(x))
    ∀x (HighIncome(x) → ¬MeansToDestination(x, bus))
    ∀x (MeansToDestination(x, bus) ⊕ MeansToDestination(x, drive))
    ∀x (HaveCar(x) → MeansToDestination(x, drive))
    ∀x (Student(x) → ¬ MeansToDestination(x, drive))
    HaveCar(james) ∨ WorkAt(james, meta)
    --------------
    CONCLUSION:
    MeansToDestination(x, drive) ∨ Student(james)
"""


prompt3 = """
    Given a set of premises and a conclusion in first order logic, your task is to analyse both and determine the logical validity of the conclusion. The logical validity can be True, False, or Uncertain.
    A True conclusion is one that can be obtained via a valid inference procedure from the given premises.
    A False conclusion is one that contradicts one or more premises during the inference procedure. 
    An Uncertain conclusion is neither True nor False. Meaning that there is insufficient information in the premises to infer it, but the conclusion it self doesn't contradict any premise.
    -------------
    The following example shows a set of premises and conclusions where each conclusion represents a different logical validity.
    FOL-PREMISES:
    ∀x (WorkAt(x, meta) → HighIncome(x))
    ∀x (HighIncome(x) → ¬MeansToDestination(x, bus))
    ∀x (MeansToDestination(x, bus) ⊕ MeansToDestination(x, drive))
    ∀x (HaveCar(x) → MeansToDestination(x, drive))
    ∀x (Student(x) → ¬ MeansToDestination(x, drive))
    HaveCar(james) ∨ WorkAt(james, meta)
    --------------
    FOL-CONCLUSION:
    MeansToDestination(x, drive) ∨ Student(james)
    Student(james)
    ¬HighIncome(james)
    --------------
    ANSWER:
    True
    False
    Uncertain
    ---------------------------
    The following is your problem:
    FOL-PREMISES
    English(thomasBarber) ∧ ProfessionalFootballer(thomasBarber)
    PlayedFor(thomasBarber, astonVilla) ∧ PlayedIn(astonVilla,theFootballLeague)
    PlayedAs(thomasBarber, halfBack) ∧ PlayedAs(thomasBarber, insideLeft)
    ScoredTheWinningGoalIn(thomasBarber, facupfinal1913)
    --------------
    FOL-CONCLUSION:
    ∃x (English(x) ∧ ProfessionalFootballer(x) ∧ ScoredTheWinningGoalIn(x, facupfinal1913))
    --------------
    ANSWER:
"""

final_prompt = """
    Given a set of premises and conclusion in first order logic, your task is to determine the logical validity of the conclusion: True, False, or Uncertain. Answer only with the logical validity.
    A True conclusion is one that can be obtained via a valid inference procedure from the given premises.
    A False conclusion is one that contradicts one or more premises during the inference procedure. 
    An Uncertain conclusion is neither True nor False. Meaning that there is insufficient information in the premises to infer it, but the conclusion it self doesn't contradict any premise.
    --------------
    FOL-PREMISES:
    ∀x (WorkAt(x, meta) → HighIncome(x))
    ∀x (HighIncome(x) → ¬MeansToDestination(x, bus))
    ∀x (MeansToDestination(x, bus) ⊕ MeansToDestination(x, drive))
    ∀x (HaveCar(x) → MeansToDestination(x, drive))
    ∀x (Student(x) → ¬ MeansToDestination(x, drive))
    HaveCar(james) ∨ WorkAt(james, meta)
    --------------
    FOL-CONCLUSION:
    MeansToDestination(x, drive) ∨ Student(james)
    Student(james)
    ¬HighIncome(james)

    Analysis:
    The first conclusion is True. Premise 6 states that either James has a car (in which case premise 4 gives us the conclusion) or James works at Meta (in which case premise 4 implies premise 2, which combined with premise 3 gives us the conclusion)
    The second conclusion is False. Premise 5 states that students can't have a Car as a MeansToDestination, however the first condition tells us James has such means.
    The third contition is Uncertain. Premise 1 is the only guarantee to have a High Income, however we can't determine that James works at Meta (Premise 6).
    ----------------------------
    
    Natural Language Premises:
    {}
    FOL Premises:
"""

### Generation Functions

In [ ]:
def chat_oss(message):
    inputs = gpt_tokenizer.apply_chat_template(
        message,
        add_generation_prompt = False,
        tokenize = True,
        return_dict = True,
        return_tensors = "pt"
    ).to(gpt_model.device)

    outputs = gpt_model.generate(**inputs, max_new_tokens = 10000) 
    # 50, 200, 350, 600, 1000 no. 2000 poquito. 
    answer = gpt_tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:-10])
    try:
        aux = re.search(r'(final<\|message\|>)', answer)
        return answer[aux.end():]
    except:
        print("No hay token de inicio del mensaje final.")


def chat_qwen(msg, thinking):
    """
        mensajes = list ; lista con el diccionario de inputs.
        thinking = bool ; valor booleano que determina si el modelo usa "thinking mode" xd
    """
    text = qwen_tokenizer.apply_chat_template(
    msg,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=thinking 
    )
    inputs = qwen_tokenizer([text], return_tensors='pt').to(dev)
    gen_ids = qwen_model.generate(**inputs, max_new_tokens = 10000) 
    # 350, 750, 1250, 1500, 2000, 4000. No </think>
    # Con 8000 apenas vimos el primter <\think> token nmms XD.
    output_ids = gen_ids[0][len(inputs.input_ids[0]):].tolist()

    try:
        index = len(gen_ids) - gen_ids[::-1].index(151668)
    except ValueError:
        index = 0

    norm_cont = qwen_tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n") 
    
    regex = re.search(r'(</think>)', norm_cont)
    try:
        return norm_cont[regex.end():] 
    except:
        print("No <\\think> token found.")


In [15]:
aux_p = final_prompt.format(dataset['premises'][0])

m_oss = [
{"role": "system", "content": "Reasoning: high."},
{"role": "developer", "content": "You translate natural language premises to first-order-logic formulars. Answer only with the translated premises."},
{"role": "user", "content": aux_p}
]

m_qwen = [
    {"role": "user", "content": aux_p}
]

response1 = chat_oss(m_oss)
response2 = chat_qwen(m_qwen, True)

print(dataset['premises'][0])
print("==========================")
print(response1)
print("==========================")
print(response2)

No professional soccer players play professional basketball.
All professional soccer defenders are professional soccer players.
All professional centerbacks are professional soccer defenders.
Some athletes are professional center-backs.
Stephen Curry plays professional basketball.
∀x (SoccerPlayer(x) → ¬BasketballPlayer(x))  
∀x (Defender(x) → SoccerPlayer(x))  
∀x (CenterBack(x) → Defender(x))  
∃x (Athlete(x) ∧ CenterBack(x))  



∀x (ProfessionalSoccerPlayer(x) → ¬PlaysProfessionalBasketball(x))  
∀x (ProfessionalSoccerDefender(x) → ProfessionalSoccerPlayer(x))  
∀x (ProfessionalCenterback(x) → ProfessionalSoccerDefender(x))  
∃x (Athlete(x) ∧ ProfessionalCenterback(x))  
PlaysProfessionalBasketball(s)


In [16]:
def translations(model):
    """
    Genera el conjunto de datos de traducción de FOLIO.
    """
    if model == 'gpt':
        message = [
        {"role": "system", "content": "Reasoning: high."},
        {"role": "developer", "content": "You translate natural language premises to first-order-logic formulars. Answer only with the translated premises."},
        {"role": "user", "content": ''}
        ]
    else:
        message = [
            {"role": "user", "content": ''}
        ]

    translation = []
    for _ in dataset["premises"]:
        aux_prompt = final_prompt.format(_)
        message[-1]['content'] = aux_prompt
        if model == 'gpt':
            answer = chat_oss(message)
        else:
            answer = chat_qwen(message, True)

        translation.append(answer)
    return translation

gpt_translations = translations('gpt')
qwen_translations =translations('qwen')

dataset['gpt Translations'] = gpt_translations
dataset['qwen Translations'] = qwen_translations

dataset.head()

,story_id,premises,premises-FOL,conclusion,conclusion-FOL,label,example_id,gpt Translations,qwen Translations
0,324,No professional soccer players play profession...,∀x ((Professional(x) ∧ SoccerPlayer(x)) → ¬Pla...,Stephen Curry is an athlete.,Athlete(stephenCurry),Uncertain,828,∀x (ProSoccerPlayer(x) → ¬PlayBasketball(x))\n...,\n\n∀x (ProfessionalSoccerPlayer(x) → ¬Profess...
1,324,No professional soccer players play profession...,∀x ((Professional(x) ∧ SoccerPlayer(x)) → ¬Pla...,Stephen Curry is an athlete and a professional...,Athlete(stephenCurry) ∧ Professional(stephenCu...,False,829,∀x (SoccerPlayer(x) → ¬BasketballPlayer(x)) \...,\n\n∀x (ProfessionalSoccerPlayer(x) → ¬PlaysBa...
2,324,No professional soccer players play profession...,∀x ((Professional(x) ∧ SoccerPlayer(x)) → ¬Pla...,"If Stephen Curry is a professional centerback,...",(Professional(stephenCurry) ∧ CenterBack(steph...,True,830,FOL Premises:\n∃x (Athlete(x) ∧ CenterBack(x))...,\n\n∀x (ProfessionalSoccerPlayer(x) → ¬Profess...
3,357,If people join student government in high scho...,"∀x (Join(x, studentGovernment) → ManyOpportuni...",Tyler joined the student government in high sc...,"JoinIn(tyler, studentGovernment, highSchool)",Uncertain,947,∀x (SG(x) → Opportunity(x)) \n∀x (Alone(x) → ...,\n\n∀x (JoinStudentGovernment(x) → Opportuniti...
4,357,If people join student government in high scho...,"∀x (Join(x, studentGovernment) → ManyOpportuni...",If Tyler either both stays alone in high schoo...,"¬(StayAloneIn(tyler, highSchool) ⊕ (Well-adjus...",True,948,∀x (JoinSG(x) → ManyOpportunities(x)) \n∀x (S...,\n\n∀x (JoinSG(x) → Opportunities(x)) \n∀x (S...


In [17]:
dataset.to_csv(r'/home/flopezp/Kurosagol/Ongoing/datasets/gpt_qwen_translations.csv')

### Deepseek

Nuevamente este modelo ~~hijo de puta~~ pesa 69,000 billones de parámetros y es lo peor del mundo tratar de usarlo desde HuggingFace. Toca usarlo desde otro lugar.

In [ ]:
from lmdeploy import pipeline

pipe = pipeline("deepseek-ai/DeepSeek-V3")

message = [{"role": "user", "content": "Tell me who Lionel Messi is."}]
output = pipe(message)
print(output)

Fetching 185 files:   0%|          | 0/185 [00:00<?, ?it/s]

configuration_deepseek.py: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

niah.png:   0%|          | 0.00/108k [00:00<?, ?B/s]

benchmark.png:   0%|          | 0.00/184k [00:00<?, ?B/s]

...

model-00023-of-000163.safetensors:   0%|          | 0.00/4.30G [00:00<?, ?B/s]

model-00021-of-000163.safetensors:   0%|          | 0.00/4.30G [00:00<?, ?B/s]

model-00024-of-000163.safetensors:   0%|          | 0.00/4.30G [00:00<?, ?B/s]

model-00025-of-000163.safetensors:   0%|          | 0.00/4.30G [00:00<?, ?B/s]

model-00026-of-000163.safetensors:   0%|          | 0.00/4.30G [00:00<?, ?B/s]